# Installation des bibliothèques nécessaires

Nous installons ici toutes les bibliothèques requises pour construire un système RAG (Retrieval Augmented Generation) basé sur Langchain, HuggingFace, FAISS, etc.

In [ ]:
pip install -q langchain torch transformers sentence-transformers datasets faiss-cpu langchain-community

: 

# Chargement du jeu de données

Nous utilisons le loader de Langchain pour charger le jeu de données `databricks/databricks-dolly-15k` depuis HuggingFace. La colonne 'context' est utilisée comme source de texte à indexer.

In [ ]:
from langchain_community.document_loaders import HuggingFaceDatasetLoader

dataset_name = "databricks/databricks-dolly-15k"
page_content_column = "context"
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)
data = loader.load()
data[:2]

: 

# Découpage des documents

Nous devons découper les documents longs en morceaux plus courts pour que les modèles puissent les traiter efficacement.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(data)
docs[0]

# Création des embeddings sémantiques

Nous utilisons `all-MiniLM-l6-v2` pour transformer les textes en vecteurs numériques qui capturent leur signification.

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

modelPath = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Exemple de test
text = "This is a test document."
embeddings.embed_query(text)[:3]

# Création du vector store FAISS

Les embeddings sont indexés dans FAISS, ce qui permet de retrouver rapidement les documents similaires à une requête.

In [ ]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

# Préparation du modèle de question-réponse

Nous utilisons ici le modèle `Intel/dynamic_tinybert`, léger et rapide, pour générer des réponses à partir du contexte récupéré.

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from langchain import HuggingFacePipeline

model_name = "Intel/dynamic_tinybert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer
)

llm = HuggingFacePipeline(
    pipeline=qa_pipeline,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

# Construction du RetrievalQA

On connecte ici la base FAISS (retriever) au modèle de génération (LLM). C’est le cœur du système RAG.

In [ ]:
from langchain.chains import RetrievalQA

retriever = db.as_retriever(search_kwargs={"k": 4})
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever)

# Test du système RAG

Nous testons enfin le système avec une requête réelle. Le modèle va retrouver les documents pertinents et générer une réponse contextualisée.

In [ ]:
question = "What is cheesemaking?"
result = qa.run({"query": question})
print(result)

# Conclusion métier

Ce système RAG permet à une entreprise de **fournir des réponses précises à des questions métiers** basées sur sa documentation interne.
**Applications concrètes** :
- FAQ intelligente (SAV)
- Assistant RH sur base documentaire
- Génération de réponses contextualisées (support interne)
- Recherche sémantique dans des documents réglementaires

Ce prototype est **modulaire** : il peut être adapté à n'importe quel jeu de données textuel de l'entreprise.